In [1]:
import json
import numpy as np
import pandas
#import os
#import io
from pprint import pprint
pandas.options.display.max_colwidth = -1

In [2]:
with open('events.json','r') as f:
    allgames = json.load(f)
print('ok')

ok


In [3]:
nEvents = 0
print('{0:,.0f} games found'.format(len(allgames)))
for game in allgames:
    nEvents = nEvents + len(game['events'])
df = pandas.DataFrame.from_dict(allgames)
print('{0:,.0f} events found'.format(nEvents))

152,259 games found
14,946,418 events found


In [21]:
#df = df.set_index('id')

#df.columns
#df.head(2)
dfx = df.sample(100)
dfx

#for index,row in df.iterrows():
#    print(index,'\n',row,'\n\n')
#df['events']


column_list = ['event_in_game', 'event_type', 'inning', 'visitor_or_home', 'batter_id', 'pitch_count', 'pitch_list']
column_list += ['event', 'comment', 'sub_batting_order', 'sub_position']

events_df = pandas.DataFrame(columns=column_list)
events_df = events_df.append(dfx['events'].iloc[0])

events_df
#print(dfx['events'].iloc[0][0])
#for index, row in dfx.iterrows():
#    for oneplay in row['events']:
#        print(oneplay,'\n')
#df['events'].head(1)#.transpose()

/home/idies/anaconda3/lib/python3.5/site-packages/pandas/core/indexes/api.py:77: RuntimeWarning: unorderable types: str() < int(), sort order is undefined for incomparable objects
  result = result.union(other)


,event_in_game,event_type,inning,visitor_or_home,batter_id,pitch_count,pitch_list,event,comment,sub_batting_order,sub_position,0
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,play
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,spand001
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,32
6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,BFBFFBX
7,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,S7/G+
8,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,
9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,


In [10]:
#weird_game_ids = ['CLE192708190', 'BOS193008310', 'BOS193107250', 'CHA193304220', 'NYA193606030', 'MLN195505152']
#weird_game_ids += ['MIN196907170', 'NYN197208160', 'KCA201109150', 'TOR201208130', 'KCA201605310']

events_log = np.empty((0,12),dtype=object)

nGames = 0
for game in allgames:     # iterate through all games 
#    if (game['id'] in weird_game_ids):  # if the game is in the list above...
    nGames = nGames + 1
    if (np.mod(nGames,1000) == 0):
        print('Processing game {0:,.0f}...'.format(nGames))
    for j in range(0,len(game['events'])):         # iterate through each event in this game
        thisevent = np.empty((1,12),dtype=object)
        thisevent[0,0] = game['id']  # save game ID first
        for k in range(1,len(game['events'][j])):
            thisevent[0,k] = game['events'][j][k-1]  # then copy all rows from a single event
        events_log = np.vstack((events_log, thisevent))   # after each event processed, append to big list

print('\n')            
print('{0:,.0f} games processed'.format(nGames))

print('Loaded into array: {0:,.0f} events'.format(len(events_log)))


# we loaded everything into a numpy array, now save it as a pandas dataframe
events_df = pandas.DataFrame(events_log)

# pandas has auto-created an index variable
events_df.index.name = 'event_id'

column_list = ['gameID', 'event_in_game', 'event_type', 'inning', 'visitor_or_home', 'batter_id', 'pitch_count', 'pitch_list']
column_list += ['event', 'comment', 'sub_batting_order', 'sub_position']
events_df.columns = column_list
#events_df
print('Done')







KeyboardInterrupt: 

In [ ]:

i = 0
while True:                   # proceed row by row through the events dataframe...
    if (i == len(events_df)):      # until the counter exceeds the length of the dataframe...
        break                        # then stop
    else:
        if (events_df['event_type'].loc[i] == 'com'):
            lastplay = i - 1
            events_df['comment'].loc[lastplay] += events_df['comment'].loc[i]
#            tester = tester.append(events_df.loc[i-1])
#            tester = tester.append(events_df.loc[i])
            while True:
#                events_df = events_df.drop(i)
                i = i + 1
                if (events_df['event_type'].loc[i] == 'com'):
                    events_df['comment'].loc[lastplay] += events_df['comment'].loc[i]
                else:
                    break
        else:
            i = i + 1

events_df = events_df[events_df['event_type'] != 'com']



In [ ]:
# renumber events in game
i = 0
i = i + 1
j = 1
while True:
#    if (np.mod(i,100) == 0):
#        print('Processing event number {0:,.0f}...'.format(i))
    if (i == len(events_df)):
        break
    else:
        events_df['event_in_game'].iloc[i] = j
        j = j + 1
        if (events_df['gameID'].iloc[i] != events_df['gameID'].iloc[i-1]):
            j = 1
    i = i + 1
    
# re-index eventID also
events_df = events_df.reset_index()

events_df[['gameID', 'event_in_game', 'event_type', 'event', 'comment']].iloc[75:95]#[events_df['gameID'] == 'CLE192708190']